In [23]:
import pandas as pd
import os
import numpy as np
import cv2
import json
import glob
import matplotlib.pyplot as plt
from sklearn.metrics import jaccard_score

In [24]:
#open manual annotated image from geojson (exported qupath image as geojason because paquo installation failed)
src = r'\\fatherserverdw\Q\research\images\skin_aging\hovernet_tile\hovernet_manual_validation'
manual = os.path.join(src,'manual_annotation')
predicted = os.path.join(src,'predicted_nuclei_contour')
raw_ims = os.path.join(src,'raw_image')

In [25]:
#open manual annotations with json.load
annotations = glob.glob(os.path.join(manual,'*.geojson'))
annotation = annotations[1]
with open(annotation) as f:
  features = json.load(f)["features"]

In [26]:
#create a list of manual contours
man_coordinates = [np.squeeze(_['geometry']['coordinates']) for _ in features]

In [27]:
manual_df = pd.DataFrame({'contour':man_coordinates})
manual_df

,contour
0,"[[368, 427], [368, 428], [367, 428], [366, 429..."
1,"[[242, 339], [241, 340], [239, 340], [238, 341..."
2,"[[168, 364], [167, 365], [166, 365], [166, 366..."
3,"[[322, 384], [321, 385], [320, 385], [319, 386..."
4,"[[387, 334], [387, 335], [386, 335], [386, 336..."
...,...
379,"[[230.0, 311.0], [230.0, 313.0], [233.0, 313.0..."
380,"[[370, 314], [370, 315], [369, 315], [369, 317..."
381,"[[959.0, 403.0], [959.0, 404.0], [957.0, 404.0..."
382,"[[543, 354], [543, 355], [542, 355], [542, 356..."


In [28]:
# def contour2centroid(c):
#     M = cv2.moments(c)
#     cX = int(M["m10"] / M["m00"])
#     cY = int(M["m01"] / M["m00"])
#     return np.array([cX,cY])

def contour2centroid(arr):
    length = arr.shape[0]
    sum_x = np.sum(arr[:, 0])
    sum_y = np.sum(arr[:, 1])
    return sum_x/length, sum_y/length

In [29]:
manual_df['centroid']=manual_df['contour'].apply(lambda row:contour2centroid(row))
manual_df

,contour,centroid
0,"[[368, 427], [368, 428], [367, 428], [366, 429...","(369.93939393939394, 438.969696969697)"
1,"[[242, 339], [241, 340], [239, 340], [238, 341...","(242.16666666666666, 346.2142857142857)"
2,"[[168, 364], [167, 365], [166, 365], [166, 366...","(172.02564102564102, 371.64102564102564)"
3,"[[322, 384], [321, 385], [320, 385], [319, 386...","(325.8695652173913, 392.4130434782609)"
4,"[[387, 334], [387, 335], [386, 335], [386, 336...","(390.0408163265306, 342.7959183673469)"
...,...,...
379,"[[230.0, 311.0], [230.0, 313.0], [233.0, 313.0...","(248.45238095238096, 313.3095238095238)"
380,"[[370, 314], [370, 315], [369, 315], [369, 317...","(390.7014925373134, 322.2089552238806)"
381,"[[959.0, 403.0], [959.0, 404.0], [957.0, 404.0...","(961.75, 413.4935897435897)"
382,"[[543, 354], [543, 355], [542, 355], [542, 356...","(547.936170212766, 360.7659574468085)"


In [30]:
#open prediction
pd_annotations = glob.glob(os.path.join(predicted,'*.json'))
#open tile11
pd_annotation = pd_annotations[0]
with open(pd_annotation) as pd_f:
  #bbox, centroid, contour
  pd_features = json.load(pd_f)["nuc"]

In [31]:
#create a list of predicted contours
pd_contours=[]
pd_centroids = []
for key in pd_features:
    temp = pd_features.get(key)
    pd_contours.append(np.array(temp.get('contour')))
    pd_centroids.append(np.round(temp.get('centroid'),0).astype('int'))

In [32]:
predicted_df = pd.DataFrame({'contour':pd_contours,'centroid':pd_centroids})
predicted_df

,contour,centroid
0,"[[667, 300], [666, 301], [666, 302], [667, 301...","[690, 305]"
1,"[[615, 306], [614, 307], [611, 307], [610, 308...","[612, 315]"
2,"[[517, 309], [516, 310], [513, 310], [513, 311...","[522, 318]"
3,"[[675, 310], [674, 311], [673, 311], [672, 312...","[679, 318]"
4,"[[503, 311], [501, 313], [499, 313], [497, 315...","[502, 318]"
...,...,...
342,"[[310, 921], [310, 922], [309, 923], [307, 923...","[310, 927]"
343,"[[698, 930], [698, 931], [697, 932], [698, 933...","[704, 938]"
344,"[[206, 942], [205, 943], [204, 943], [203, 944...","[206, 945]"
345,"[[331, 951], [327, 955], [327, 956], [326, 957...","[331, 958]"


In [33]:
#get size of origianl image
raw = glob.glob(os.path.join(raw_ims,'*.tif'))
raw_img = cv2.imread(raw[0])
dim = raw_img.shape
image_size = (dim[0],dim[1])

In [34]:
#Link ground truth to predicted nuclei
from sklearn.neighbors import NearestNeighbors

#create a list of manual centroids, and a list of predicted centroids
manual_centroid= np.array(manual_df['centroid'].values.tolist())
predicted_centroid= np.array(predicted_df['centroid'].values.tolist())
nbrs = NearestNeighbors(n_neighbors=1, metric='euclidean').fit(predicted_centroid) #build model
distances, indices = nbrs.kneighbors(manual_centroid) #apply model
indices=np.squeeze(indices)

In [35]:
N_fp = len(predicted_df)-len(np.unique(indices))
N_fp

17

In [ ]:
#a list of paired predicted contours
paired_pred_contour=[]
for idx in np.unique(indices):
    paired_pred_contour.append(predicted_df.iloc[idx].contour)


In [119]:
#false positive indexes
# (but we also need to add the ones with low iou,we don't know if it's a false positive or false negative

unpaired_predicted_idx = [_ for _ in np.array(predicted_df.index) if _ not in np.unique(indices)]

In [120]:
#a list of unpaired predicted contours
unpaired_pred_contour=[] #we need to add more
for idx in unpaired_predicted_idx:
    unpaired_pred_contour.append(predicted_df.iloc[idx].contour)


17

In [78]:
#a list of paired predicted contours
paired_pred_contour=[]
for idx in np.unique(indices):
    paired_pred_contour.append(predicted_df.iloc[idx].contour)



In [16]:
predicted_df2 = predicted_df.iloc[indices]
predicted_df2

,contour,centroid
104,"[[369, 428], [367, 430], [367, 431], [366, 432...","[370, 439]"
22,"[[242, 338], [241, 339], [239, 339], [238, 340...","[242, 346]"
33,"[[172, 363], [171, 364], [169, 364], [168, 365...","[173, 372]"
57,"[[321, 384], [320, 385], [319, 385], [316, 388...","[325, 392]"
20,"[[391, 334], [390, 335], [388, 335], [387, 336...","[390, 343]"
...,...,...
22,"[[242, 338], [241, 339], [239, 339], [238, 340...","[242, 346]"
6,"[[360, 314], [359, 315], [357, 315], [356, 316...","[377, 320]"
77,"[[957, 405], [956, 406], [954, 406], [952, 408...","[960, 413]"
26,"[[545, 354], [544, 355], [542, 355], [540, 357...","[549, 361]"


In [17]:
linked_df = manual_df
linked_df['pred_contour'] =predicted_df2.contour.reset_index(drop=True)
linked_df['pred_centroid'] =predicted_df2.centroid.reset_index(drop=True)
linked_df

,contour,centroid,pred_contour,pred_centroid
0,"[[368, 427], [368, 428], [367, 428], [366, 429...","(369.93939393939394, 438.969696969697)","[[369, 428], [367, 430], [367, 431], [366, 432...","[370, 439]"
1,"[[242, 339], [241, 340], [239, 340], [238, 341...","(242.16666666666666, 346.2142857142857)","[[242, 338], [241, 339], [239, 339], [238, 340...","[242, 346]"
2,"[[168, 364], [167, 365], [166, 365], [166, 366...","(172.02564102564102, 371.64102564102564)","[[172, 363], [171, 364], [169, 364], [168, 365...","[173, 372]"
3,"[[322, 384], [321, 385], [320, 385], [319, 386...","(325.8695652173913, 392.4130434782609)","[[321, 384], [320, 385], [319, 385], [316, 388...","[325, 392]"
4,"[[387, 334], [387, 335], [386, 335], [386, 336...","(390.0408163265306, 342.7959183673469)","[[391, 334], [390, 335], [388, 335], [387, 336...","[390, 343]"
...,...,...,...,...
379,"[[230.0, 311.0], [230.0, 313.0], [233.0, 313.0...","(248.45238095238096, 313.3095238095238)","[[242, 338], [241, 339], [239, 339], [238, 340...","[242, 346]"
380,"[[370, 314], [370, 315], [369, 315], [369, 317...","(390.7014925373134, 322.2089552238806)","[[360, 314], [359, 315], [357, 315], [356, 316...","[377, 320]"
381,"[[959.0, 403.0], [959.0, 404.0], [957.0, 404.0...","(961.75, 413.4935897435897)","[[957, 405], [956, 406], [954, 406], [952, 408...","[960, 413]"
382,"[[543, 354], [543, 355], [542, 355], [542, 356...","(547.936170212766, 360.7659574468085)","[[545, 354], [544, 355], [542, 355], [540, 357...","[549, 361]"


In [18]:
scores=[]
#label is the pixel value (color) from [1,384]
for index, row in linked_df.iterrows():
    groundtruth_mask = np.zeros(image_size)
    predicted_mask = np.zeros(image_size)
    groundtruth_mask = cv2.fillPoly(groundtruth_mask, pts=[np.array(row['contour']).astype(np.int32)],  color=1)
    predicted_mask = cv2.fillPoly(predicted_mask, pts=[np.array(row['pred_contour']).astype(np.int32)],  color=1)
    score = jaccard_score(groundtruth_mask,predicted_mask, average="micro")
    scores.append(score)

KeyboardInterrupt: 

In [ ]:
linked_df['jaccard_score']=scores
linked_df #Look into why some cells have low score when centroids actually match

In [ ]:
plt.hist(linked_df['jaccard_score'],100) #groundtruth has roughly 37 more nuclei 384 vs 347

In [ ]:
np.mean(linked_df['jaccard_score'])

In [ ]:
def get_fast_pq(true, pred, match_iou=0.5):
    """`match_iou` is the IoU threshold level to determine the pairing between
    GT instances `p` and prediction instances `g`. `p` and `g` is a pair
    if IoU > `match_iou`. However, pair of `p` and `g` must be unique
    (1 prediction instance to 1 GT instance mapping).

    If `match_iou` < 0.5, Munkres assignment (solving minimum weight matching
    in bipartite graphs) is caculated to find the maximal amount of unique pairing.

    If `match_iou` >= 0.5, all IoU(p,g) > 0.5 pairing is proven to be unique and
    the number of pairs is also maximal.

    Returns:
        [dq, sq, pq]: measurement statistic

        [paired_true, paired_pred, unpaired_true, unpaired_pred]:
                      pairing information to perform measurement

    """
    assert match_iou >= 0.0, "Cant' be negative"

    true = np.copy(true)
    pred = np.copy(pred)
    true_id_list = list(np.unique(true))
    pred_id_list = list(np.unique(pred))

    true_masks = [
        None,
    ]
    for t in true_id_list[1:]:
        t_mask = np.array(true == t, np.uint8)
        true_masks.append(t_mask)

    pred_masks = [
        None,
    ]
    for p in pred_id_list[1:]:
        p_mask = np.array(pred == p, np.uint8)
        pred_masks.append(p_mask)

    # prefill with value
    pairwise_iou = np.zeros(
        [len(true_id_list) - 1, len(pred_id_list) - 1], dtype=np.float64
    )

    # caching pairwise iou
    for true_id in true_id_list[1:]:  # 0-th is background
        t_mask = true_masks[true_id]
        pred_true_overlap = pred[t_mask > 0]
        pred_true_overlap_id = np.unique(pred_true_overlap)
        pred_true_overlap_id = list(pred_true_overlap_id)
        for pred_id in pred_true_overlap_id:
            if pred_id == 0:  # ignore
                continue  # overlaping background
            p_mask = pred_masks[pred_id]
            total = (t_mask + p_mask).sum()
            inter = (t_mask * p_mask).sum()
            iou = inter / (total - inter)
            pairwise_iou[true_id - 1, pred_id - 1] = iou
    #
    if match_iou >= 0.5:
        paired_iou = pairwise_iou[pairwise_iou > match_iou]
        pairwise_iou[pairwise_iou <= match_iou] = 0.0
        paired_true, paired_pred = np.nonzero(pairwise_iou)
        paired_iou = pairwise_iou[paired_true, paired_pred]
        paired_true += 1  # index is instance id - 1
        paired_pred += 1  # hence return back to original
    else:  # * Exhaustive maximal unique pairing
        #### Munkres pairing with scipy library
        # the algorithm return (row indices, matched column indices)
        # if there is multiple same cost in a row, index of first occurence
        # is return, thus the unique pairing is ensure
        # inverse pair to get high IoU as minimum
        paired_true, paired_pred = linear_sum_assignment(-pairwise_iou)
        ### extract the paired cost and remove invalid pair
        paired_iou = pairwise_iou[paired_true, paired_pred]

        # now select those above threshold level
        # paired with iou = 0.0 i.e no intersection => FP or FN
        paired_true = list(paired_true[paired_iou > match_iou] + 1)
        paired_pred = list(paired_pred[paired_iou > match_iou] + 1)
        paired_iou = paired_iou[paired_iou > match_iou]

    # get the actual FP and FN
    unpaired_true = [idx for idx in true_id_list[1:] if idx not in paired_true]
    unpaired_pred = [idx for idx in pred_id_list[1:] if idx not in paired_pred]
    # print(paired_iou.shape, paired_true.shape, len(unpaired_true), len(unpaired_pred))

    #
    tp = len(paired_true)
    fp = len(unpaired_pred)
    fn = len(unpaired_true)
    # get the F1-score i.e DQ
    dq = tp / (tp + 0.5 * fp + 0.5 * fn)
    # get the SQ, no paired has 0 iou so not impact
    sq = paired_iou.sum() / (tp + 1.0e-6)

    return [dq, sq, dq * sq], [paired_true, paired_pred, unpaired_true, unpaired_pred]

In [ ]:
def get_fast_aji_plus(true, pred):
    """AJI+, an AJI version with maximal unique pairing to obtain overall intersecion.
    Every prediction instance is paired with at most 1 GT instance (1 to 1) mapping, unlike AJI
    where a prediction instance can be paired against many GT instances (1 to many).
    Remaining unpaired GT and Prediction instances will be added to the overall union.
    The 1 to 1 mapping prevents AJI's over-penalisation from happening.

    Fast computation requires instance IDs are in contiguous orderding i.e [1, 2, 3, 4]
    not [2, 3, 6, 10]. Please call `remap_label` before hand and `by_size` flag has no
    effect on the result.

    """
    true = np.copy(true)  # ? do we need this
    pred = np.copy(pred)
    true_id_list = list(np.unique(true))
    pred_id_list = list(np.unique(pred))

    true_masks = [
        None,
    ]
    for t in true_id_list[1:]:
        t_mask = np.array(true == t, np.uint8)
        true_masks.append(t_mask)

    pred_masks = [
        None,
    ]
    for p in pred_id_list[1:]:
        p_mask = np.array(pred == p, np.uint8)
        pred_masks.append(p_mask)

    # prefill with value
    pairwise_inter = np.zeros(
        [len(true_id_list) - 1, len(pred_id_list) - 1], dtype=np.float64
    )
    pairwise_union = np.zeros(
        [len(true_id_list) - 1, len(pred_id_list) - 1], dtype=np.float64
    )

    # caching pairwise
    for true_id in true_id_list[1:]:  # 0-th is background
        t_mask = true_masks[true_id]
        pred_true_overlap = pred[t_mask > 0]
        pred_true_overlap_id = np.unique(pred_true_overlap)
        pred_true_overlap_id = list(pred_true_overlap_id)
        for pred_id in pred_true_overlap_id:
            if pred_id == 0:  # ignore
                continue  # overlaping background

            p_mask = pred_masks[pred_id]
            total = (t_mask + p_mask).sum()
            inter = (t_mask * p_mask).sum()
            pairwise_inter[true_id - 1, pred_id - 1] = inter
            pairwise_union[true_id - 1, pred_id - 1] = total - inter
    #
    pairwise_iou = pairwise_inter / (pairwise_union + 1.0e-6)
    #### Munkres pairing to find maximal unique pairing
    paired_true, paired_pred = linear_sum_assignment(-pairwise_iou)
    ### extract the paired cost and remove invalid pair
    paired_iou = pairwise_iou[paired_true, paired_pred]
    # now select all those paired with iou != 0.0 i.e have intersection
    paired_true = paired_true[paired_iou > 0.0]
    paired_pred = paired_pred[paired_iou > 0.0]
    paired_inter = pairwise_inter[paired_true, paired_pred]
    paired_union = pairwise_union[paired_true, paired_pred]
    paired_true = list(paired_true + 1)  # index to instance ID
    paired_pred = list(paired_pred + 1)
    overall_inter = paired_inter.sum()
    overall_union = paired_union.sum()


    # add all unpaired GT and Prediction into the union
    unpaired_true = np.array(
        [idx for idx in true_id_list[1:] if idx not in paired_true]
    )
    unpaired_pred = np.array(
        [idx for idx in pred_id_list[1:] if idx not in paired_pred]
    )
    for true_id in unpaired_true:
        overall_union += true_masks[true_id].sum()
    for pred_id in unpaired_pred:
        overall_union += pred_masks[pred_id].sum()
    #
    aji_score = overall_inter / overall_union
    return aji_score


In [19]:
#test, compute metrics for paried nucleus
#add the score for each pair tp dataframe

#dice_1
dice1 = []

#pq
paired_true=0
unpaired_true=0
paired_iou = []
#threshold for iou
match_iou = 0.5

#aji
paired_inter= []
paired_union = []
unpaired_true_arr = []

for index, row in linked_df.iterrows():
    groundtruth_mask = np.zeros(image_size)
    predicted_mask = np.zeros(image_size)
    groundtruth_mask = cv2.fillPoly(groundtruth_mask, pts=[np.array(row['contour']).astype(np.int32)],  color=1)
    predicted_mask = cv2.fillPoly(predicted_mask, pts=[np.array(row['pred_contour']).astype(np.int32)],  color=1)

    #inter1 = groundtruth_mask * predicted_mask
    #denom = groundtruth_mask + predicted_mask
    #dice = 2.0 * np.sum(inter1) / np.sum((denom+1.0e-6))
    #dice1.append(dice)


    iou = jaccard_score(groundtruth_mask,predicted_mask, average="micro")
    if(iou>0):
        total = ( groundtruth_mask + predicted_mask).sum()
        inter = ( groundtruth_mask * predicted_mask).sum()
        paired_inter.append(inter)
        paired_union.append(total-inter)
    else:
        #record unpaired_true
        unpaired_true_arr.append(groundtruth_mask)

    #iou is the same  as jaccard_score

    #if(iou >= match_iou):
        #paired_iou.append(iou)
        #paired_true = paired_true+1
    #extract unpaired nucleus
    #else:
        #unpaired_true  = unpaired_true+1






#Panoptic Quality
tp = paired_true
fp = len(predicted_df)-len(np.unique(indices))
fn = unpaired_true
    # get the F1-score i.e DQ
dq = tp / (tp + 0.5 * fp + 0.5 * fn)
    # get the SQ, no paired has 0 iou so not impact
sq = sum(paired_iou) / (tp + 1.0e-6)

#Panoptic Quality and aji is for the whole image
pq = dq * sq

print(dq,sq,pq)

#aji
overall_inter = paired_inter.sum()
overall_union = paired_union.sum()

#TODO: add unpaired to overall union
#add unpaired true
for up_true in unpaired_true_arr:
    overall_union += up_true.sum()
#add unpaired predicted

#aji score
aji_score = overall_inter / overall_union

KeyboardInterrupt: 

In [ ]:
#add metrics to dataframe
#calculate aji
linked_df['dice1']=dice1
linked_df


In [ ]:
#ashley:try out the metrics, inputs are binary masks of groundtruth and predicted nucleus
#their functions also convert the true and pred inputs to binary masks.
# I think they just take in an integer array of an image

#question: do you compute these scores for each nuclei with its pair and take an average, or do you do it on the overall image
#i did it on the overall image
dc1= get_dice_1(groundtruth_mask,predicted_mask)
dc2 = get_fast_dice_2(groundtruth_mask,predicted_mask) #this equals dc1

dq, sq, pq = get_fast_pq(groundtruth_mask,predicted_mask)[0]


#TODO: need to modify the function because the function is expecting labeled masks
aji = get_fast_aji_plus(groundtruth_mask,predicted_mask)

#theyalso have a pair-coordinates function

In [21]:
np.unique(indices)

array([  0,   1,   2,   3,   5,   6,   7,   8,  10,  12,  13,  14,  15,
        16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,
        30,  31,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,
        44,  45,  46,  47,  48,  49,  50,  51,  52,  54,  55,  56,  57,
        58,  59,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,
        72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,
        85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
        98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110,
       111, 112, 113, 114, 115, 116, 117, 118, 119, 121, 122, 123, 124,
       126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138,
       139, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152,
       153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165,
       166, 168, 169, 170, 171, 172, 174, 175, 176, 177, 178, 179, 180,
       181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 19

In [ ]:
#don't know what this does
paired_true, paired_pred, unpaired_true, unpaired_pred = get_fast_pq(groundtruth_mask,predicted_mask)[1]

In [ ]:
np.mean(linked_df['jaccard_score'])

In [ ]:
def get_fast_pq(true, pred, match_iou=0.5):
    """`match_iou` is the IoU threshold level to determine the pairing between
    GT instances `p` and prediction instances `g`. `p` and `g` is a pair
    if IoU > `match_iou`. However, pair of `p` and `g` must be unique
    (1 prediction instance to 1 GT instance mapping).

    If `match_iou` < 0.5, Munkres assignment (solving minimum weight matching
    in bipartite graphs) is caculated to find the maximal amount of unique pairing.

    If `match_iou` >= 0.5, all IoU(p,g) > 0.5 pairing is proven to be unique and
    the number of pairs is also maximal.

    Returns:
        [dq, sq, pq]: measurement statistic

        [paired_true, paired_pred, unpaired_true, unpaired_pred]:
                      pairing information to perform measurement

    """
    assert match_iou >= 0.0, "Cant' be negative"

    true = np.copy(true)
    pred = np.copy(pred)
    true_id_list = list(np.unique(true))
    pred_id_list = list(np.unique(pred))

    true_masks = [
        None,
    ]
    for t in true_id_list[1:]:
        t_mask = np.array(true == t, np.uint8)
        true_masks.append(t_mask)

    pred_masks = [
        None,
    ]
    for p in pred_id_list[1:]:
        p_mask = np.array(pred == p, np.uint8)
        pred_masks.append(p_mask)

    # prefill with value
    pairwise_iou = np.zeros(
        [len(true_id_list) - 1, len(pred_id_list) - 1], dtype=np.float64
    )

    # caching pairwise iou
    for true_id in true_id_list[1:]:  # 0-th is background
        t_mask = true_masks[true_id]
        pred_true_overlap = pred[t_mask > 0]
        pred_true_overlap_id = np.unique(pred_true_overlap)
        pred_true_overlap_id = list(pred_true_overlap_id)
        for pred_id in pred_true_overlap_id:
            if pred_id == 0:  # ignore
                continue  # overlaping background
            p_mask = pred_masks[pred_id]
            total = (t_mask + p_mask).sum()
            inter = (t_mask * p_mask).sum()
            iou = inter / (total - inter)
            pairwise_iou[true_id - 1, pred_id - 1] = iou
    #
    if match_iou >= 0.5:
        paired_iou = pairwise_iou[pairwise_iou > match_iou]
        pairwise_iou[pairwise_iou <= match_iou] = 0.0
        paired_true, paired_pred = np.nonzero(pairwise_iou)
        paired_iou = pairwise_iou[paired_true, paired_pred]
        paired_true += 1  # index is instance id - 1
        paired_pred += 1  # hence return back to original
    else:  # * Exhaustive maximal unique pairing
        #### Munkres pairing with scipy library
        # the algorithm return (row indices, matched column indices)
        # if there is multiple same cost in a row, index of first occurence
        # is return, thus the unique pairing is ensure
        # inverse pair to get high IoU as minimum
        paired_true, paired_pred = linear_sum_assignment(-pairwise_iou)
        ### extract the paired cost and remove invalid pair
        paired_iou = pairwise_iou[paired_true, paired_pred]

        # now select those above threshold level
        # paired with iou = 0.0 i.e no intersection => FP or FN
        paired_true = list(paired_true[paired_iou > match_iou] + 1)
        paired_pred = list(paired_pred[paired_iou > match_iou] + 1)
        paired_iou = paired_iou[paired_iou > match_iou]

    # get the actual FP and FN
    unpaired_true = [idx for idx in true_id_list[1:] if idx not in paired_true]
    unpaired_pred = [idx for idx in pred_id_list[1:] if idx not in paired_pred]
    # print(paired_iou.shape, paired_true.shape, len(unpaired_true), len(unpaired_pred))

    #
    tp = len(paired_true)
    fp = len(unpaired_pred)
    fn = len(unpaired_true)
    # get the F1-score i.e DQ
    dq = tp / (tp + 0.5 * fp + 0.5 * fn)
    # get the SQ, no paired has 0 iou so not impact
    sq = paired_iou.sum() / (tp + 1.0e-6)

    return [dq, sq, dq * sq], [paired_true, paired_pred, unpaired_true, unpaired_pred]

In [ ]:
def get_fast_aji_plus(true, pred):
    """AJI+, an AJI version with maximal unique pairing to obtain overall intersecion.
    Every prediction instance is paired with at most 1 GT instance (1 to 1) mapping, unlike AJI
    where a prediction instance can be paired against many GT instances (1 to many).
    Remaining unpaired GT and Prediction instances will be added to the overall union.
    The 1 to 1 mapping prevents AJI's over-penalisation from happening.

    Fast computation requires instance IDs are in contiguous orderding i.e [1, 2, 3, 4]
    not [2, 3, 6, 10]. Please call `remap_label` before hand and `by_size` flag has no
    effect on the result.

    """
    true = np.copy(true)  # ? do we need this
    pred = np.copy(pred)
    true_id_list = list(np.unique(true))
    pred_id_list = list(np.unique(pred))

    true_masks = [
        None,
    ]
    for t in true_id_list[1:]:
        t_mask = np.array(true == t, np.uint8)
        true_masks.append(t_mask)

    pred_masks = [
        None,
    ]
    for p in pred_id_list[1:]:
        p_mask = np.array(pred == p, np.uint8)
        pred_masks.append(p_mask)

    # prefill with value
    pairwise_inter = np.zeros(
        [len(true_id_list) - 1, len(pred_id_list) - 1], dtype=np.float64
    )
    pairwise_union = np.zeros(
        [len(true_id_list) - 1, len(pred_id_list) - 1], dtype=np.float64
    )

    # caching pairwise
    for true_id in true_id_list[1:]:  # 0-th is background
        t_mask = true_masks[true_id]
        pred_true_overlap = pred[t_mask > 0]
        pred_true_overlap_id = np.unique(pred_true_overlap)
        pred_true_overlap_id = list(pred_true_overlap_id)
        for pred_id in pred_true_overlap_id:
            if pred_id == 0:  # ignore
                continue  # overlaping background

            p_mask = pred_masks[pred_id]
            total = (t_mask + p_mask).sum()
            inter = (t_mask * p_mask).sum()
            pairwise_inter[true_id - 1, pred_id - 1] = inter
            pairwise_union[true_id - 1, pred_id - 1] = total - inter
    #
    pairwise_iou = pairwise_inter / (pairwise_union + 1.0e-6)
    #### Munkres pairing to find maximal unique pairing
    paired_true, paired_pred = linear_sum_assignment(-pairwise_iou)
    ### extract the paired cost and remove invalid pair
    paired_iou = pairwise_iou[paired_true, paired_pred]
    # now select all those paired with iou != 0.0 i.e have intersection
    paired_true = paired_true[paired_iou > 0.0]
    paired_pred = paired_pred[paired_iou > 0.0]
    paired_inter = pairwise_inter[paired_true, paired_pred]
    paired_union = pairwise_union[paired_true, paired_pred]
    paired_true = list(paired_true + 1)  # index to instance ID
    paired_pred = list(paired_pred + 1)
    overall_inter = paired_inter.sum()
    overall_union = paired_union.sum()


    # add all unpaired GT and Prediction into the union
    unpaired_true = np.array(
        [idx for idx in true_id_list[1:] if idx not in paired_true]
    )
    unpaired_pred = np.array(
        [idx for idx in pred_id_list[1:] if idx not in paired_pred]
    )
    for true_id in unpaired_true:
        overall_union += true_masks[true_id].sum()
    for pred_id in unpaired_pred:
        overall_union += pred_masks[pred_id].sum()
    #
    aji_score = overall_inter / overall_union
    return aji_score


In [98]:
#test, compute metrics for paried nucleus
#add the score for each pair tp dataframe

#dice_1
dice1 = []

#pq
paired_true=0
unpaired_true=0
paired_iou = []
#threshold for iou
match_iou = 0.5

#aji
paired_inter= []
paired_union = []


for index, row in linked_df.iterrows():
    groundtruth_mask = np.zeros(image_size)
    predicted_mask = np.zeros(image_size)
    groundtruth_mask = cv2.fillPoly(groundtruth_mask, pts=[np.array(row['contour']).astype(np.int32)],  color=1)
    predicted_mask = cv2.fillPoly(predicted_mask, pts=[np.array(row['pred_contour']).astype(np.int32)],  color=1)

    #inter1 = groundtruth_mask * predicted_mask
    #denom = groundtruth_mask + predicted_mask
    #dice = 2.0 * np.sum(inter1) / np.sum((denom+1.0e-6))
    #dice1.append(dice)


    iou = jaccard_score(groundtruth_mask,predicted_mask, average="micro")
    if(iou>0):
        total = ( groundtruth_mask + predicted_mask).sum()
        inter = ( groundtruth_mask * predicted_mask).sum()
        paired_inter.append(inter)
        paired_union.append(total-inter)


    #iou is the same  as jaccard_score

    #if(iou >= match_iou):
        #paired_iou.append(iou)
        #paired_true = paired_true+1
    #extract unpaired nucleus
    #else:
        #unpaired_true  = unpaired_true+1






#Panoptic Quality
tp = paired_true
fp = len(predicted_df)-len(np.unique(indices))
fn = unpaired_true
    # get the F1-score i.e DQ
dq = tp / (tp + 0.5 * fp + 0.5 * fn)
    # get the SQ, no paired has 0 iou so not impact
sq = sum(paired_iou) / (tp + 1.0e-6)

#Panoptic Quality and aji is for the whole image
pq = dq * sq

print(dq,sq,pq)

#aji
overall_inter = paired_inter.sum()
overall_union = paired_union.sum()

#add unpaired to overall union

0.8656294200848657 0.7460312648556475


306

78

In [78]:
#add metrics to dataframe
#calculate aji
linked_df['dice1']=dice1
linked_df


,contour,centroid,pred_contour,pred_centroid,jaccard_score,iou,dice1
0,"[[368, 427], [368, 428], [367, 428], [366, 429...","(369.93939393939394, 438.969696969697)","[[369, 428], [367, 430], [367, 431], [366, 432...","[370, 439]",0.776923,0.776923,0.872570
1,"[[242, 339], [241, 340], [239, 340], [238, 341...","(242.16666666666666, 346.2142857142857)","[[242, 338], [241, 339], [239, 339], [238, 340...","[242, 346]",0.890152,0.890152,0.940000
2,"[[168, 364], [167, 365], [166, 365], [166, 366...","(172.02564102564102, 371.64102564102564)","[[172, 363], [171, 364], [169, 364], [168, 365...","[173, 372]",0.854305,0.854305,0.919786
3,"[[322, 384], [321, 385], [320, 385], [319, 386...","(325.8695652173913, 392.4130434782609)","[[321, 384], [320, 385], [319, 385], [316, 388...","[325, 392]",0.815873,0.815873,0.897033
4,"[[387, 334], [387, 335], [386, 335], [386, 336...","(390.0408163265306, 342.7959183673469)","[[391, 334], [390, 335], [388, 335], [387, 336...","[390, 343]",0.821429,0.821429,0.900489
...,...,...,...,...,...,...,...
379,"[[230.0, 311.0], [230.0, 313.0], [233.0, 313.0...","(248.45238095238096, 313.3095238095238)","[[242, 338], [241, 339], [239, 339], [238, 340...","[242, 346]",0.000000,0.000000,0.000000
380,"[[370, 314], [370, 315], [369, 315], [369, 317...","(390.7014925373134, 322.2089552238806)","[[360, 314], [359, 315], [357, 315], [356, 316...","[377, 320]",0.541069,0.541069,0.701606
381,"[[959.0, 403.0], [959.0, 404.0], [957.0, 404.0...","(961.75, 413.4935897435897)","[[957, 405], [956, 406], [954, 406], [952, 408...","[960, 413]",0.730627,0.730627,0.843450
382,"[[543, 354], [543, 355], [542, 355], [542, 356...","(547.936170212766, 360.7659574468085)","[[545, 354], [544, 355], [542, 355], [540, 357...","[549, 361]",0.890977,0.890977,0.940476


In [52]:
#ashley:try out the metrics, inputs are binary masks of groundtruth and predicted nucleus
#their functions also convert the true and pred inputs to binary masks.
# I think they just take in an integer array of an image

#question: do you compute these scores for each nuclei with its pair and take an average, or do you do it on the overall image
#i did it on the overall image
dc1= get_dice_1(groundtruth_mask,predicted_mask)
dc2 = get_fast_dice_2(groundtruth_mask,predicted_mask) #this equals dc1

dq, sq, pq = get_fast_pq(groundtruth_mask,predicted_mask)[0]


#TODO: need to modify the function because the function is expecting labeled masks
aji = get_fast_aji_plus(groundtruth_mask,predicted_mask)

#theyalso have a pair-coordinates function

In [46]:
#don't know what this does
paired_true, paired_pred, unpaired_true, unpaired_pred = get_fast_pq(groundtruth_mask,predicted_mask)[1]